In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
from torch import Tensor
import torch.optim as optim

from testing import logit_diff_metric
from applications.pipeline import run_attribution_steps, identify_target_components, optimise_edit_components
from applications.datasets import CounterFact

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device

In [5]:
device = get_device()
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


In [26]:
# Verify that loading works, for one example

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))

print(clean_input)
print(corrupted_input)
print(labels)

clean_tokens = model.to_tokens(clean_input)
corrupted_tokens = model.to_tokens(corrupted_input)

clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = logit_diff_metric(clean_logits, labels)
print(f"Clean logit difference: {clean_logit_diff}")

corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)
corrupted_logit_diff = logit_diff_metric(corrupted_logits, labels)
print(f"Corrupted logit difference: {corrupted_logit_diff}")

['The mother tongue of Danielle Darrieux is']
['The mother tongue of Paul McCartney is']
tensor([[24111, 15823]])
Clean logit difference: tensor([0.1160], device='cuda:0', grad_fn=<SubBackward0>)
Corrupted logit difference: tensor([-1.1990], device='cuda:0', grad_fn=<SubBackward0>)


In [25]:
n_epochs = 5

original_tokens = model.to_tokens(clean_input)
rewrite_tokens = model.to_tokens(corrupted_input)

original_logits, original_cache = model.run_with_cache(original_tokens)
original_logit_diff = logit_diff_metric(original_logits, labels)

rewrite_logits, rewrite_cache = model.run_with_cache(rewrite_tokens)
rewrite_logit_diff = logit_diff_metric(rewrite_logits, labels)

# LOCALISATION STAGE

mlp_highlighted, attn_highlighted = run_attribution_steps(
    model,
    original_tokens,
    rewrite_tokens,
    labels,
    original_cache,
    rewrite_cache,
    original_logit_diff,
    rewrite_logit_diff,
)

target_mlp = identify_target_components(model, mlp_highlighted)
target_attn = identify_target_components(model, attn_highlighted)

# EDITING STAGE

relevant_parameters = [
    p for name, p in model.named_parameters() if "attn" in name or "mlp" in name
]
optimiser = optim.Adam(relevant_parameters, lr=2e-4)

for _ in range(n_epochs):
    logits = model(original_tokens)
    answer_index = labels[:, 1]  # Aim for rewritten answer
    optimise_edit_components(
        model, logits, answer_index, target_mlp, target_attn, optimiser
    )

AssertionError: Baseline can be provided as a tensor for just one input and broadcasted to the batch or input and baseline must have the same shape or the baseline corresponding to each input tensor must be a scalar. Found baseline: tensor([[[[ 6.3183e-02,  5.9828e-02, -1.8017e-02,  ..., -3.1300e-02,
            1.3576e-01,  1.0932e-01],
          [ 1.8581e-01,  2.3438e-01, -2.4526e-01,  ..., -5.3240e-01,
           -3.8492e-01,  2.4711e-01],
          [-4.0007e-02, -1.3616e-01,  3.8954e-01,  ..., -6.7756e-02,
            1.2671e-01, -1.8432e-01],
          ...,
          [-2.8296e-01, -5.0378e-01,  1.6165e-01,  ..., -4.9117e-02,
           -1.7141e-01, -1.1625e-01],
          [-5.6480e-02, -4.1342e-01, -2.0710e-01,  ...,  1.9242e-01,
            2.5598e-01, -7.1256e-02],
          [ 1.5511e-01, -5.1378e-01, -1.7323e-01,  ...,  1.5156e-01,
           -3.0645e-01,  3.7273e-01]],

         [[ 5.6135e-02,  5.2765e-02,  8.7760e-03,  ..., -4.0619e-02,
            1.3522e-01,  9.8149e-02],
 